In [1190]:
import pandas as pd
import numpy as np
import random
from datetime import time, datetime

In [1191]:
# GingerbreadStore Class

class GingerbreadStore():
    ###
    # registration_form layout: (store_name - address_string - opening_hours_string - manager - capital)
    ###
    def __init__(self, registration_form):
        self.registration_form = registration_form
        self.store_name = None
        self.address_string = None
        self.address = None
        self.manager = None
        self.opening_hours_string = None
        self.opening_hours = None
        self.stock = {}
        self.capital_bod = 0
        self.capital = 0
        
        self.prices = {
            'blank': (1.9, 0.6),
            'egg_glaze': (2.5, 0.8),
            'dried_fruit': (2.6, 0.8),
            'white_chocolate': (3.2, 1.4),
            'milk_chocolate': (3.2, 1.4),
            'dark_chocolate': (3.2, 1.4),
            'raspberry': (5.0, 2.5)
        }
        
        
    # extract data from registration form
    def data_extracting(self):
        split_data = self.registration_form.split(' - ')
        
        self.store_name = split_data[0]
        self.address_string = split_data[1]
        self.opening_hours_string = split_data[2]
        self.manager = split_data[3]
        self.capital_bod = float(split_data[4])
        self.capital = float(split_data[4])
        
    
    # helper function for updating opening_hours from opening_hours_string
    def update_opening_hours(self):
        self.opening_hours = {}
        
        # weekdays
        weekdays = []
        if 'mon' in self.opening_hours_string:
            weekdays.append(0)
        if 'tue' in self.opening_hours_string:
            weekdays.append(1)
        if 'wed' in self.opening_hours_string:
            weekdays.append(2)
        if 'thu' in self.opening_hours_string:
            weekdays.append(3)
        if 'fri' in self.opening_hours_string:
            weekdays.append(4)
        if 'sat' in self.opening_hours_string:
            weekdays.append(5)
        if 'sun' in self.opening_hours_string:
            weekdays.append(6)
            
        self.opening_hours['weekdays'] = weekdays
            
        # hours
        hours_string = self.opening_hours_string.rsplit(' ', 1)[1]
        hours = hours_string.split('-', 1)
        hours_from = hours[0].split(':', 1)
        hours_to = hours[1].split(':', 1)
        
        self.opening_hours['from'] = time(hour=int(hours_from[0]), minute=int(hours_from[1]))
        self.opening_hours['to'] = time(hour=int(hours_to[0]), minute=int(hours_to[1]))
        
        
    # update address from address_string and opening_hours from opening_hours_string
    def data_featuring(self):
        # address
        self.address = {}
        self.address['number'] = int(self.address_string.split(' ', 1)[0])
        self.address['street'] = self.address_string.split(' ', 1)[1].rsplit(' ', 1)[0]
        self.address['postal code'] = int(self.address_string.rsplit(' ', 1)[1])
        
        # opening hours
        self.update_opening_hours()

        
    # get dict with store data
    def get_data_cleaned(self):
        clean = {}
        
        clean['store_name'] = self.store_name
        clean['address'] = self.address
        clean['address_string'] = self.address_string
        clean['manager'] = self.manager
        clean['opening_hours'] = self.opening_hours
        clean['opening_hours_string'] = self.opening_hours_string
        clean['prices'] = self.prices
        clean['stock'] = self.stock
        clean['capital_bod'] = self.capital_bod
        clean['capital'] = self.capital
        
        return clean
    

    # buy <amount> items of gingerbread <kind>
    def buy(self, kind, amount):
        # check if gingerbread kind is available for buying
        if not kind in self.prices.keys():
            raise Exception('Gingerbread kind is not available.')
        # check if capital is sufficient to buy gingerbread
        if (amount*self.prices[kind][1] > self.capital):
            raise Exception('Not enough money available.')
        # decorated and stale gingerbread is not for sale
        if 'decorated' in kind or 'stale' in kind:
            return
            
        # add gingerbread to stock
        if not kind in self.stock:
            self.stock[kind] = 0
        self.stock[kind] += amount
        # substract price from capital
        self.capital -= amount*self.prices[kind][1]
        
    
    # sell <amount> items of gingerbread <kind>
    def sell(self, kind, amount):
        # check if gingerbread kind is available for sale
        if not kind in self.prices.keys():
            raise Exception('Gingerbread kind is not available.')
        # check if stock is sufficient to sell requested amount
        if (amount > self.stock[kind]):
            raise Exception('Not enough stock.')
            
        # substract gingerbread from stock
        self.stock[kind] -= amount
        # add price to capital
        self.capital += amount*self.prices[kind][0]
        

    # check if store is open at a given point_in_time
    def is_open(self, point_in_time):
        if point_in_time.weekday() in self.opening_hours['weekdays']:
            if point_in_time.time() >= self.opening_hours['from']:
                if point_in_time.time() <= self.opening_hours['to']:
                    return True
        return False


    # check if store has gingerbread kind
    def has_gingerbread_kind(self, kind):
        if kind in self.stock.keys() and self.stock[kind] != 0:
            return True
        return False
    

    # decorate gingerbread to increase the price
    def decorate_gingerbread(self, kind, amount):
        # check if gingerbread kind is available for decorating
        if not kind in self.stock.keys():
            raise Exception('Gingerbread kind is not in stock.')
        # check if stock is decorate to sell requested amount
        if (amount > self.stock[kind]):
            raise Exception('Not enough gingerbread to decorate.')
        
        if 'decorated' in kind:
            kind_decorated = kind
        else:
            kind_decorated = kind + '_decorated'

        # decorating increases the sales price by 50%
        decorated_tuple = (self.prices[kind][0]*1.5, 0)
        self.prices[kind_decorated] = decorated_tuple
        
        # move decorated gingerbread in stock
        if not kind_decorated in self.stock:
            self.stock[kind_decorated] = 0
        self.stock[kind_decorated] += amount
        self.stock[kind] -= amount
        
        # decorating costs 0.2€ per gingerbread
        self.capital -= 0.2*amount
    

    # letting gingerbread go stale decreases the price
    def let_gingerbread_go_stale(self, kind, amount):
        if not kind in self.stock.keys():
            raise Exception('Gingerbread kind is not in stock.')
        if (amount > self.stock[kind]):
            raise Exception('Not enough gingerbread to decorate.')
        
        if 'stale' in kind:
            kind_stale = kind
        else:
            kind_stale = kind + '_stale'

        # letting the gingerbread go stale decreases the sales price by 20%
        stale_tuple = (self.prices[kind][0]*0.8, 0)
        self.prices[kind_stale] = stale_tuple
        
        # move stale gingerbread in stock
        if not kind_stale in self.stock:
            self.stock[kind_stale] = 0
        self.stock[kind_stale] += amount
        self.stock[kind] -= amount
        

In [1198]:
# Functions

# returns a list of stores open at a given point_in_time
def find_open_stores(stores, point_in_time):
    open_stores = []
    for sto in stores:
        if sto.is_open(point_in_time):
            open_stores.append(sto.store_name)
    
    return open_stores


# returns a list of stores that have gingerbread kind
def find_gingerbread(stores, kind):
    available_stores = []
    for sto in stores:
        if sto.has_gingerbread_kind(kind):
            available_stores.append(sto.store_name)
    
    return available_stores


# returns a list of cleaned stores data
def get_stores_data_cleaned(stores):
    cleaned_factory = []
    for sto in stores:
        cleaned_factory.append(sto.get_data_cleaned())
    return cleaned_factory


# returns DataFrame with the end-of-day-calculations, including revenue and total remaining stock
def eod_calculation(stores):
    cleaned_factory = get_stores_data_cleaned(stores)
    df = pd.DataFrame.from_records(cleaned_factory, columns=['store_name', 'prices', 'stock', 'capital_bod', 'capital'])

    total_stock = []
    revenues = []

    for sto in cleaned_factory:
        # calculate revenue through current capital and capital at beginning-of-day
        revenues.append(sto['capital'] - sto['capital_bod'])
        # calculate total remaining stock
        total = 0
        for kind in sto['stock']:
            total += sto['stock'][kind]
        total_stock.append(total)

    df['revenue'] = revenues
    df['total_stock'] = total_stock

    return df

# returns DataFrame with store information
def stores_info(stores):
    cleaned_factory = get_stores_data_cleaned(stores)
    df = pd.DataFrame.from_records(cleaned_factory, columns=['store_name', 'manager', 'address_string', 'address', 'opening_hours_string', 'opening_hours', 'capital_bod'])
    return df


# awards the store with the highest revenue with <amount> added to capital
def award_bestseller(stores, amount):
    data = eod_calculation(stores)
    column = data['revenue']

    max_index = column.idxmax()
    stores[max_index].capital += amount

    print('Congratulations', stores[max_index].store_name, '!')
    print('You have been awarded', amount, '€. Your capital now stands at', stores[max_index].capital, '€.')
    return
    

In [1199]:
# Stores & Factory creation

crumb_path = GingerbreadStore('The Crumb Path - 18 rue de sully 75002 - mon tue wed thu fri 9:30-17:00 - Celine Dion - 10000')
sugar_rush = GingerbreadStore('Sugar Rush - 20 rue de gentillese 75001 - mon thu fri 8:00-15:30 - Marie Curie - 15000')
sweeter_bread = GingerbreadStore('Sweeter Bread - 159 rue chatelet 75005 - sat sun 9:00-16:30 - Edith Piaf - 2000')
ginger_spice = GingerbreadStore('The Ginger Spice - 2 rue de paramont 75015 - tue wed thu fri sat 9:00-19:00 - Audrey Tautou - 20000')
honey_honey = GingerbreadStore('Honey Honey - 9 rue du point 75242 - thu fri sat sun 10:00-16:30 - Zaz - 12000')
grace_jelly = GingerbreadStore('Grace Jelly - 31 rue fragolle 75004 - mon tue thu 8:00-16:00 - Mika - 6000')

gingerbread_factory = [crumb_path, sugar_rush, sweeter_bread, ginger_spice, honey_honey, grace_jelly]

for store in gingerbread_factory:
    store.data_extracting()
    store.data_featuring()


In [1200]:
# Stores activity - can be executed multilple times to simulate activity

for store in gingerbread_factory:

    # gingerbread kinds available at store in the beginning
    available_gingerbread_kinds = [x for x in store.prices] 

    for kind in available_gingerbread_kinds:
        try:
            # buy random amount of kind if fewer than 50 are currently in stock
            if (not kind in store.stock.keys()) or store.stock[kind] < 50:
                store.buy(kind, random.randint(0,10)*10)

            # 50/50 chance whether random amount of kind in stock is decorated
            if bool(random.getrandbits(1)):
                store.decorate_gingerbread(kind, random.randint(0,store.stock[kind]))
            # 50/50 chance whether random amount of kind in stock goes stale
            if bool(random.getrandbits(1)):
                store.let_gingerbread_go_stale(kind, random.randint(0,store.stock[kind]))
        except Exception as e:
            print(e)
            continue

    for kind in store.stock.keys(): 
        try:
            # sell random amount of kind in stock
            store.sell(kind, random.randint(0,store.stock[kind]))
        except Exception as e:
            print(e)
            continue

In [1195]:
# End of Day Calculation

display(eod_calculation(gingerbread_factory))

,store_name,prices,stock,capital_bod,capital,revenue,total_stock
0,The Crumb Path,"{'blank': (1.9, 0.6), 'egg_glaze': (2.5, 0.8),...","{'blank': 0, 'blank_stale': 0, 'egg_glaze': 1,...",10000.0,10325.26,325.26,122
1,Sugar Rush,"{'blank': (1.9, 0.6), 'egg_glaze': (2.5, 0.8),...","{'blank': 7, 'blank_decorated': 0, 'blank_stal...",15000.0,15198.40,198.40,111
2,Sweeter Bread,"{'blank': (1.9, 0.6), 'egg_glaze': (2.5, 0.8),...","{'blank': 8, 'blank_decorated': 1, 'egg_glaze'...",2000.0,1906.46,-93.54,176
3,The Ginger Spice,"{'blank': (1.9, 0.6), 'egg_glaze': (2.5, 0.8),...","{'blank': 1, 'blank_stale': 31, 'egg_glaze': 5...",20000.0,19944.64,-55.36,252
4,Honey Honey,"{'blank': (1.9, 0.6), 'egg_glaze': (2.5, 0.8),...","{'blank': 4, 'blank_stale': 30, 'egg_glaze': 4...",12000.0,12252.64,252.64,193
5,Grace Jelly,"{'blank': (1.9, 0.6), 'egg_glaze': (2.5, 0.8),...","{'blank': 9, 'egg_glaze': 12, 'egg_glaze_stale...",6000.0,5996.28,-3.72,111


In [1196]:
# Award best-selling store

award_bestseller(gingerbread_factory, 1000)

Congratulations The Crumb Path !
You have been awarded 1000 €. Your capital now stands at 11325.26 €.


In [1201]:
# Random requests

# currently open stores
print('Stores currently open:', find_open_stores(gingerbread_factory, datetime.now()))

# stores that have decorated raspberry gingerbread
print('Stores that have decorated raspberry gingerbread:', find_gingerbread(gingerbread_factory, 'raspberry_decorated'))

# stores that have no stale egg_glaze gingerbread
stale_stores = find_gingerbread(gingerbread_factory, 'egg_glaze_stale')
store_names = []
for sto in gingerbread_factory:
    store_names.append(sto.store_name)
print('Stores that have no stale egg glaze gingerbread:', list(set(store_names) - set(stale_stores)))

display(stores_info(gingerbread_factory))

Stores currently open: ['The Crumb Path', 'The Ginger Spice']
Stores that have decorated raspberry gingerbread: ['The Ginger Spice', 'Honey Honey']
Stores that have no stale egg glaze gingerbread: ['Honey Honey', 'The Ginger Spice']


,store_name,manager,address_string,address,opening_hours_string,opening_hours,capital_bod
0,The Crumb Path,Celine Dion,18 rue de sully 75002,"{'number': 18, 'street': 'rue de sully', 'post...",mon tue wed thu fri 9:30-17:00,"{'weekdays': [0, 1, 2, 3, 4], 'from': 09:30:00...",10000.0
1,Sugar Rush,Marie Curie,20 rue de gentillese 75001,"{'number': 20, 'street': 'rue de gentillese', ...",mon thu fri 8:00-15:30,"{'weekdays': [0, 3, 4], 'from': 08:00:00, 'to'...",15000.0
2,Sweeter Bread,Edith Piaf,159 rue chatelet 75005,"{'number': 159, 'street': 'rue chatelet', 'pos...",sat sun 9:00-16:30,"{'weekdays': [5, 6], 'from': 09:00:00, 'to': 1...",2000.0
3,The Ginger Spice,Audrey Tautou,2 rue de paramont 75015,"{'number': 2, 'street': 'rue de paramont', 'po...",tue wed thu fri sat 9:00-19:00,"{'weekdays': [1, 2, 3, 4, 5], 'from': 09:00:00...",20000.0
4,Honey Honey,Zaz,9 rue du point 75242,"{'number': 9, 'street': 'rue du point', 'posta...",thu fri sat sun 10:00-16:30,"{'weekdays': [3, 4, 5, 6], 'from': 10:00:00, '...",12000.0
5,Grace Jelly,Mika,31 rue fragolle 75004,"{'number': 31, 'street': 'rue fragolle', 'post...",mon tue thu 8:00-16:00,"{'weekdays': [0, 1, 3], 'from': 08:00:00, 'to'...",6000.0
